# Introduction to Dimensionality Reduction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from sklearn.decomposition import PCA
from scipy.stats import zscore
import pandas as pd

#### A walk on the beach

<img src="beach.jpeg" width="600">
<div style="text-align: center"> source: https://www.publicdomainpictures.net/en/free-download.php?image=shadows-on-the-beach&id=177457 </div>

#### Looking for repetition: a recipe book

<img src="recipes.png" width="700">

<img src="recipes2.png" width="1000">

#### Nonlinearities during dimensionality reduction: calculating body mass index

The body mass index (BMI) is calculated form a person's height and weight as an indicator of body composition. The BMI is calculated as follows:

$$
BMI = \frac{weight}{height^2}
$$

The units of BMI are therefore $m / kg^2$. Calculating BMI has the benefit of reducing two variables into a single variable, amounting to dimensionality reduction.

Now, could we say that calculating BMI amounts to *linear* dimensionality reduction?

How about the following?

$$
\begin{align}
l.BMI = log(BMI) &= log(\frac{weight}{height^2})\\
&= log(weight) - log(height^2)\\
&= log(weight) - 2log(height)\\
&= l.weight - l.height\\
\\
l.BMI &= l.weight - l.height
\end{align}
$$

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1,2, figsize=[7,3.5])

heights = np.linspace(1.5, 2.1, 100)
weights = np.linspace(50, 120, 100)
hv, wv = np.meshgrid(heights, weights)
bmisv = wv / hv**2
hwb = np.stack([hv.flatten(), wv.flatten(), bmisv.flatten()]).T

log_heights = np.log(np.linspace(1.5, 2.1, 100))
log_weights = np.log(np.linspace(50, 120, 100))
lhv, lwv = np.meshgrid(log_heights, log_weights)
log_bmisv = lwv - 2*lhv
log_hwb = np.stack([lhv.flatten(), lwv.flatten(), log_bmisv.flatten()]).T

s0 = ax[0].contourf(hv, wv, bmisv); ax[0].set_xlabel('height (m)'); ax[0].set_ylabel('weight (kg)')
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar0 = fig.colorbar(s0, cax=cax, orientation='vertical')
cbar0.ax.set_title('BMI ($kg/m^2$)')

s1 = ax[1].contourf(lhv, lwv, log_bmisv); ax[1].set_xlabel('log height (m)'); ax[1].set_ylabel('log weight (kg)')
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar1 = fig.colorbar(s1, cax=cax, orientation='vertical')
cbar1.ax.set_title('log BMI ($kg/m^2$)')

plt.tight_layout(); plt.show()

### Principal Components Analysis (PCA)

<img src="pca.png" width="800">
<div style="text-align: center"> source: https://www.publicdomainpictures.net/en/free-download.php?image=shadows-on-the-beach&id=177457 </div>

In [ ]:
## Import iris dataset
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
## Store data in pandas DataFrame
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

iris_df['target'] = iris_df['target'].map({0:iris.target_names[0], 1:iris.target_names[1], 2:iris.target_names[2]})
iris_df.rename(columns = {'target':'species'}, inplace=True)
iris_df

In [ ]:
## Extract numerical values in arrays
x = iris_df.iloc[:,:-1].values

In [ ]:
## Normalize data: zero mean & unit variance
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x)
x_trunc = x[:,:-1]  ## first three features, for visualization

In [ ]:
## Visualize truncated data containing first three features
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d", elev=-150, azim=110)
ax.scatter(x_trunc[:,0], x_trunc[:,1], x_trunc[:,2]);

In [ ]:
## Compare 2D projections of full vs truncated data

fig, ax = plt.subplots(1,1,figsize=[6.,6.])
ax.set_xlabel('PCA 1'); ax.set_ylabel('PCA 2')
ax.set_xticks([]); ax.set_yticks([])

## 2D PCA projection of truncated data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x_trunc)
ax.scatter(x_2d[:,0], x_2d[:,1])

## 2D PCA projection of full data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x)
ax.scatter(x_2d[:,0], x_2d[:,1])

plt.show()

In [ ]:
## Compare "manual" 2D projection to automatic 2D projection

fig, ax = plt.subplots(1,1,figsize=[6.,6.])
ax.set_xlabel('PCA 1'); ax.set_ylabel('PCA 2')
ax.set_xticks([]); ax.set_yticks([])

## 2D PCA projection of full data
pca = PCA(n_components=2)
x_2d = pca.fit_transform(x)
ax.scatter(x_2d[:,0], x_2d[:,1])

## 2D PCA projection of full data - matrix multiplication
x_2d_manual = x @ pca.components_.T
ax.scatter(x_2d_manual[:,0], x_2d_manual[:,1]);

plt.show()

In [ ]:
## Show 2D projection alongside weights for different features

fig, ax = plt.subplots(1,2,figsize=[12.,5.])
ax[0].set_xlabel('PCA 1 ({:.2})'.format(pca.explained_variance_ratio_[0]))  # proportion of variance explained by PCA component 1
ax[0].set_ylabel('PCA 2 ({:.2})'.format(pca.explained_variance_ratio_[1]))  # proportion of variance explained by PCA component 2
ax[0].set_xticks([]); ax[0].set_yticks([])

x_2d_manual = x @ pca.components_.T
ax[0].scatter(x_2d_manual[:,0], x_2d_manual[:,1]);

## Barplot of components' weights
bp = pd.DataFrame(pca.components_, columns=iris_df.columns[:-1].str.strip(' (cm)'), index=['PCA 1', 'PCA 2']).plot.bar(ax=ax[1], rot=0);
bp.set_ylabel('weights');
for p in ax[1].patches:
    ax[1].annotate(str(round(p.get_height(), 2)), (p.get_x()-0.005, p.get_height()*1.025))

In [ ]:
## "Manual" 2D projection - project dimensions individually

xi = x[0,:]
print(xi)

xi_1 = (xi * pca.components_[0,:]).sum()  # project onto 1st component
xi_2 = (xi * pca.components_[1,:]).sum()  # project onto 2nd component

print(np.float32([xi_1, xi_2]))
print(np.float32(x_2d[0,:]))

#### Kernel PCA and Probabilistic PCA

PCA provides *linear* components which can be unsuitable for data with non-linear structures. Non-linear functions called *kernels* can be used to represent data such that PCA can provide suitable linear components of that non-linear representation. This method is termed **kernel PCA**.

<img src="kpca.png" width="900">
<div style="text-align: center"> source: https://www.cs.mcgill.ca/~dprecup/courses/ML/Lectures/ml-lecture13.pdf </div>

PCA does not attempt to model noise which accompany its components. In **probabilistic PCA**, a noise variance term $\sigma^2$ accounts for random variations within its components, allowing projections against its components with varying spreads. Probabilistic PCA can thus model both the linear structure and noise components of its components. Due to its probabilistic nature, probabilistic PCA can also be used as a generative model.

<img src="ppca.png" width="1000">
<div style="text-align: center"> source: https://www.youtube.com/watch?v=lJ0cXPoEozg </div>

#### Independent Component Analysis (ICA)

PCA provides *orthogonal* components / coordinates, which can limit the modelling capacities of PCA. On the other hand, such ICA components are not constrained to be orthogonal which provides ICA with more "flexibility".

<img src="pca_vs_ica.png" width="800">
<div style="text-align: center"> source: https://brainvision.com/applications/eeg/ </div>

#### Electroencephalography (EEG) data analysis
 
 The EEG is a brain recording modality where electrodes are placed on the scalp/head to measure fluctuations in bioelectical potentials. Hence, every electrode measures a time-varying signal.
 
  <img src="eeg_image.png" width="400">
  <div style="text-align: center"> source: https://brainvision.com/applications/eeg/ </div>

Import EEG data using [MNE Python](https://mne.tools/stable/index.html) analysis package. We will be using open-source data which has been reported in:

[Babayan, A., Erbey, M., Kumral, D. et al. *A mind-brain-body dataset of MRI, EEG, cognition, emotion, and peripheral physiology in young and old adults.* Sci Data 6, 180308 (2019).](https://www.nature.com/articles/sdata2018308)

In [ ]:
## Load EEG data
eeg = mne.io.read_raw_eeglab("sub-010321_EC_downsamp.set")
eeg.annotations.delete( np.arange( len(eeg.annotations.description) ) )  # remove annotations, not important

In [ ]:
## Plot EEG electrode layout and EEG signals
eeg.plot_sensors(show_names=True, sphere=90);
eeg.plot(n_channels=12, duration=4);  # recommend plotting using "notebook" plotting backend, i.e. %matplotlib notebook

From the time-series plots, it seems that many time-series are alike. Perhaps there's a way to find patterns and have a more succinct representation of our data. One way to find these patterns is to use **Independent Components Analysis** (ICA) on our EEG data.

  <img src="eeg_ica.jpeg" width="500">
  <div style="text-align: center"> source: https://sccn.ucsd.edu/~jung/Site/EEG_artifact_removal.html </div>

In [ ]:
## Compute ICA on EEG data
num_comps = 15  # choose number of patterns, or "components"

ica = ICA(n_components=num_comps, random_state=97)
ica.fit(eeg)

In [ ]:
## Plot ICA components
ica.plot_sources(eeg, start=10, stop=16);
ica.plot_components();

In [ ]:
## Compute PCA on EEG data
num_comps = 15  # choose number of patterns, or "components"

eeg_ts = eeg.get_data().T
pca = PCA(n_components=num_comps)
pca_data_ts = pca.fit_transform(eeg_ts)

In [ ]:
## Plot PCA components
len_plot = 1000     # choose time-series length for plotting purposes

num_plots = pca_data_ts.shape[1]
plt.figure(figsize=[9.,9.])

for i in range(num_plots):
    ax = plt.subplot( num_plots , 1, i+1 )
    plt.plot(zscore(pca_data_ts)[:len_plot,i])
    
    ax.set_ylabel("PCA {}".format(i), fontsize=8, rotation=0)
    ax.set_xticks([]); ax.set_yticks([])
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    
plt.tight_layout()
plt.show()

### Nonlinear dimensionality reduction: t-SNE and UMAP

Linear dimensionality reduction methods such as PCA may be incapable of properly displaying data in a low-dimensional space (e.g. 2D). As previously discussed, kernel PCA leverages nonlinear transformations (i.e. kernels) such that the resulting representation is ammenable to standard PCA. However, kernel PCA requires a pre-defined kernel function to obtain a suitable nonlinear representation.

We will now explore two nonlinear dimensionality reduction methods which do not require such pre-defined kernels. These are:
1. t-distributed Stochastic Neighbor Embedding (**t-SNE**)
2. Uniform Manifold Approximation and Projection (**UMAP**)

t-SNE and UMAP are often used in the analysis of single-cell RNA sequencing (scRNA-seq) data which is very high dimensional, e.g. d = 20'000 genes. scRNA-seq data from the mouse prancreas will be used here for demonstration.

In [ ]:
import pickle
import scanpy as sc
import random
import pandas as pd
import anndata
import matplotlib.pyplot as plt

In [ ]:
## Import and format data

random.seed(10)

# mouse pancreas single-cell dataset
# read in data and cell type labels
with open('MP.pickle', 'rb') as f:
    df = pickle.load(f)

with open('MP_genes.pickle', 'rb') as f:
    genes = pickle.load(f)

df.set_index('Unnamed: 0', inplace=True)  # set first column (cell ID as the index column)
sample_id = pickle.load(open('cell_IDs.pkl', 'rb'))
df = df.loc[list(sample_id), :]

X = df[genes].values  # extract the N x M cells-by-genes matrix

sample_info = pd.read_csv('sample_info.csv')

mp_anndata = anndata.AnnData(X=X)
mp_anndata.obs['Celltype'] = sample_info['assigned_cluster'].values
celltype_one_hot = pd.get_dummies(mp_anndata.obs['Celltype'], prefix='Celltype').values

N = X.shape[0]  # number of single-cell samples
K = len(sample_info['assigned_cluster'].unique())  # number of topics
M = X.shape[1]  # number of genes

mp_anndata

In [ ]:
## Compute and plot 2D t-SNE projection
_, ax = plt.subplots(figsize=(7, 6))

sc.tl.tsne(mp_anndata, perplexity=50)
sc.pl.tsne(mp_anndata, color=["Celltype"], ax=ax, title='two-dimensional tSNE plot')

In [ ]:
## Compute neighbor affinities
sc.pp.neighbors(mp_anndata, n_neighbors=30)

## Compute and plot 2D UMAP projection
_, ax = plt.subplots(figsize=(7, 6))

sc.tl.umap(mp_anndata, min_dist=0.1)
sc.pl.umap(mp_anndata, color=["Celltype"], ax=ax, title='two-dimensional UMAP plot')

### Autoencoders
Adapted from: https://www.theaidream.com/post/an-introduction-to-autoencoder-and-variational-autoencoder-vae

In [ ]:
import numpy as np
import keras
from keras import layers
from keras.datasets import mnist
import matplotlib.pyplot as plt

Vanilla AE

In [ ]:
# This is the size of our encoded representations
encoding_dim = 2  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
hidden_dim = 64

# This is our input image
input_img = keras.Input(shape=(784,))

# "encoded" is the encoded representation of the input
hidden_enc = layers.Dense(hidden_dim, activation='relu')(input_img)
encoded = layers.Dense(encoding_dim, activation='relu')(hidden_enc)
ae_encoder = keras.Model(input_img, encoded, name='encoder')

# "decoded" is the lossy reconstruction of the input
encoded_inputs = keras.Input(shape=(encoding_dim,), name='z_sampling')
hidden_dec = layers.Dense(hidden_dim, activation='relu')(encoded_inputs)
decoded = layers.Dense(784, activation='sigmoid')(hidden_dec)
ae_decoder = keras.Model(encoded_inputs, decoded, name='decoder')

output_img = ae_decoder(ae_encoder(input_img))

autoencoder = keras.Model(input_img, output_img, name='ae')

#Now let's train our autoencoder to reconstruct MNIST digits.
#First, we'll configure our model to use a per-pixel binary crossentropy loss, and the Adam optimizer:

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

#Let's prepare our input data. We're using MNIST digits, and we're discarding the labels (since we're only interested in encoding/decoding the input images).

(x_train, _), (x_test, y_test) = mnist.load_data()

#We will normalize all values between 0 and 1 and we will flatten the 28x28 images into vectors of size 784.

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

#Now let's train our autoencoder for 50 epochs:
autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, x_test))

#After 50 epochs, the autoencoder seems to reach a stable train/validation loss value of about 0.09. We can try to visualize the reconstructed inputs and the encoded representations. We will use Matplotlib.

# Encode and decode some digits
# Note that we take them from the *test* set

encoded_imgs = ae_encoder.predict(x_test)
decoded_imgs = ae_decoder.predict(encoded_imgs)
n = 10  # Number of digits to display
plt.figure(figsize=(20, 4))

for i in range(n):
    # Display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

x_test_encoded = ae_encoder.predict(x_test, verbose=0)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:,0], x_test_encoded[:,1], c=y_test, cmap='tab10')
plt.colorbar()
plt.show()

In [ ]:
# Display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
# We will sample n points within [-15, 15] standard deviations
grid_x = np.flip(np.linspace(0, 120, n))
grid_y = np.linspace(0, 100, n)
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = ae_decoder.predict(z_sample, verbose=0)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit
plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()

Variational AE

In [ ]:
#First, here's our encoder network, mapping inputs to our latent distribution parameters:
original_dim = 28 * 28
intermediate_dim = 64
latent_dim = 2

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

#We can use these parameters to sample new similar points from the latent space:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

#Finally, we can map these sampled latent points back to reconstructed inputs:
# Create encoder
vae_encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)

vae_decoder = keras.Model(latent_inputs, outputs, name='decoder')

# Instantiate VAE model
outputs = vae_decoder(vae_encoder(inputs)[2])

vae = keras.Model(inputs, outputs, name='vae_mlp')

#We train the model using the end-to-end model, with a custom loss function: the sum of a reconstruction term, and the KL divergence regularization term.

reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

vae.compile(optimizer='adam')

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


#We train our VAE on MNIST digits:
vae.fit(x_train, x_train,
        epochs=10,
        batch_size=32,
        validation_data=(x_test, x_test))

In [ ]:
batch_size = 32

x_test_encoded = vae_encoder.predict(x_test, verbose=0)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[0][:,0], x_test_encoded[0][:,1], c=y_test, cmap='tab10')
plt.colorbar()
plt.show()

In [ ]:
# Display a 2D manifold of the digits
n = 15  # figure with 15x15 digits
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

# We will sample n points within [-15, 15]
grid_x = np.flip(np.linspace(-3, 3, n))
grid_y = np.linspace(-3, 3, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = vae_decoder.predict(z_sample, verbose=0)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit
plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()

### Other topics in dimensionality reduction

1. Dimensionality reduction via latent associations between datasets
2. Tensor decompositions
3. Dimensinality reduction of autocorrelated data
4. Categorical and mixed data

1. Latent associations between two datasets

#### Sandbox